# Perform read smoothing then assemble with LJA

In [12]:
%run "Header.ipynb"
%run "../main-workflow/utils.py"

In [2]:
import os
import time
import pickle
import pysam
import skbio
from collections import defaultdict, Counter
from linked_mutations_utils import find_mutated_positions

## 0. Quick sanity check: ensure that all $k$-mers ($k$ = 5,001) are unique in each MAG

In [3]:
# I know there are actual k-mer counting tools you can use but no reason to overcomplicate things for now

k = 5001

for seq in SEQS:
    fasta = skbio.DNA.read(f"../seqs/{seq}.fasta")
    bargain_bin_kmer_counter = Counter()
    
    # The skbio.DNA object is 0-indexed, so 0 is the leftmost k-mer start position and
    # ((seq length) - k) is the rightmost k-mer start position. The + 1 is because python ranges don't include
    # the right endpoint.
    for start_pos in range(0, seq2len[seq] - k + 1):
        
        # NOTE: this is a terrible no good very bad way to do this; it's more efficient to use a "sliding window"
        # approach where you store the entire k-mer and then, with each step, just remove the first character and
        # add on a new last character. "But, uh, this code will only be run on these three MAGs, so I'm gonna
        # prioritize clarity over optimization," says me, the insane person who just spent like a minute writing
        # this comment when I could've been optimizing this code instead look WHATEVER this counts k-mers and it's
        # 4am let's not overcomplicate it, look if you're on GitHub right now and you see this inane comment
        # we can both just pretend that you were looking at some really optimized code and we'll both walk away
        # satisfied, capisce
        kmer = fasta[start_pos : start_pos + k]
        
        bargain_bin_kmer_counter[str(kmer)] += 1
        if start_pos % 1000000 == 0: print(f"On start pos {start_pos:,} in {seq2name[seq]}.")
    
    mckc = bargain_bin_kmer_counter.most_common(1)[0][1]
    print(f"The most common k = {k:,}-mer in {seq2name[seq]} occurred {mckc:,} time(s).")

On start pos 0 in CAMP.
On start pos 1,000,000 in CAMP.
The most common k = 5,001-mer in CAMP occurred 1 time(s).
On start pos 0 in BACT1.
On start pos 1,000,000 in BACT1.
On start pos 2,000,000 in BACT1.
The most common k = 5,001-mer in BACT1 occurred 1 time(s).
On start pos 0 in BACT2.
On start pos 1,000,000 in BACT2.
On start pos 2,000,000 in BACT2.
The most common k = 5,001-mer in BACT2 occurred 1 time(s).


## 1. Smooth reads

Lots of this code is duplicated from the `Phasing-01-MakeGraph.ipynb` notebook in this folder.

In [4]:
# Set this to True to actually generate ordinary smoothed reads that include called mutations;
# set this to False to generate "sanity check" perfect smoothed reads, where no mutations are included
# and the read entirely matches the reference
actually_include_mutations_in_the_smoothed_reads = True

In [5]:
# We'll need to know the mean coverage of each sequence when computing virtual reads.
seq2meancov = get_meancovs()
seq2meancov

Sequence edge_6104 has average coverage 4,158.57 and median coverage 4,122.00.
Sequence edge_1671 has average coverage 1,415.07 and median coverage 1,436.00.
Sequence edge_2358 has average coverage 2,993.46 and median coverage 2,936.00.


{'edge_6104': 4158.572468826692,
 'edge_1671': 1415.072755380576,
 'edge_2358': 2993.461913625056}

In [6]:
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")
output_dir = "phasing-data/smoothed-reads/"

# verbose?
no_indoor_voice = False

def write_out_reads(filepath, readname2seq):
    # Notably, this uses the "a" (append) method in order to add to the end of a file
    with open(filepath, "a") as of:
        for readname in readname2seq:
            # Write out both the header and the sequence for each read
            of.write(f">{readname}\n{str(readname2seq[readname])}\n")
            
ALN_UPDATE_FREQ = 5000
ALN_BUFFER_FREQ = 1000
VR_EXTRA_SPAN = 100

P = 1
            
t1 = time.time()
for seq in SEQS:
    
    # Record which positions (0-indexed) aren't covered by any smoothed reads in this MAG.
    # We'll add "virtual reads" that span these positions.
    uncovered_positions = set(range(0, seq2len[seq]))
    
    fasta = skbio.DNA.read(f"../seqs/{seq}.fasta")
    
    output_smoothed_reads_file = os.path.join(output_dir, f"{seq}_smoothed_reads.fasta")
    
    # Identify all (0-indexed, so compatible with skbio / pysam!)
    # mutated positions in this genome up front to save time.
    #
    # Equivalently, we could also just take in an arbitrary VCF as input
    # (e.g. one produced from another variant calling tool), although we'd
    # need to be careful to only include SNVs and not indels/etc...
    
    print("=" * 70)
    print(f"Identifying mutated positions (p = {P}%) in genome {seq2name[seq]}...")
    mutpos2pileup = find_mutated_positions(seq, p_to_use=P, incl_pileup=True)
    # We sort because the code below relies on these being in ascending order
    mutated_positions = sorted(mutpos2pileup.keys())
    print(f"Found {len(mutated_positions):,} mutated positions (p = {P}%) in {seq2name[seq]}.")
    print(
        f"Note that this tally is higher than you'd see in e.g. the CP1/2/3 plots, because now we're including "
        "both 'rare' and non-rare mutations. Just so you don't waste five minutes sanity-checking this like I did."
    )
    
    print("Going through these positions...")
    
    # Instead of just writing out every smoothed alignment as soon as we generate it, we build up a "buffer"
    # of these alignments and then write a bunch out at once. This way we limit slowdown due to constantly
    # having to open/close files. I don't really have a good source for this as best practice, but I remembered
    # to do it while writing this code, so somewhere in College Park the CS faculty at Maryland are smiling
    #
    # Also fyi this maps read name to smoothed alignment (well, at this point, just read) sequence. The read name
    # is useful to preserve in fasta files so we have some idea of provenance (where smoothed reads came from)
    smoothed_aln_buffer = {}
    
    # The first time we see an alignment of a read, it's 1; if we see a supp aln of this read, it's 2; etc.
    # Lets us distinguish alignments with different names
    readname2freq_so_far = defaultdict(int)
    
    # Go through all linear alignments of each read to this genome, focusing (for now) on just the primary
    # alignments...
    ts1 = time.time()
    for ai, aln in enumerate(bf.fetch(seq), 1):
        
        if ai % ALN_UPDATE_FREQ == 0:
            print(
                f"\tOn aln {ai:,} in seq {seq2name[seq]}. "
                f"Time spent on {seq2name[seq]} so far: {time.time() - ts1:,.2f} sec."
            )
            
        if aln.is_secondary:
            raise ValueError(
                "Not to get political or anything, but you should've already filtered secondary alns out"
            )
            
        # Note that supplementary alignments are ok, though! We implicitly handle these here.
        #
        # Different alignments of the same read will have different new_readnames, because we're gonna
        # be treating them as distinct "reads". We should have already filtered reference-overlapping
        # supp alns so this shouldn't be a problem
        
        readname = aln.query_name
        readname2freq_so_far[readname] += 1
        new_readname = f"{readname}_{readname2freq_so_far[readname]}"
        
        # should never happen
        if new_readname in smoothed_aln_buffer:
            raise ValueError("This exact read alignment has already been smoothed? Weird.")
            
        # Figure out where on the MAG this alignment "hits." These are 0-indexed positions from Pysam.
        # (reference_end points to the position after the actual final position, since these are designed to
        # be interoperable with Python's half-open intervals.)
        #
        # Of course, there likely will be indels within this range: we're purposefully ignoring those here.
        ref_start = aln.reference_start
        ref_end = aln.reference_end - 1
        
        # This should never happen (TM)
        if ref_start >= ref_end:
            # Du sollst jetzt mit Gott sprechen.
            raise ValueError(
                f"Ref start {ref_start:,} >= ref end {ref_end:,} for read {new_readname}?"
            )
            
        # Record which positions this read covers (of course, it may not exactly "cover" these positions
        # originally due to indels, but the smoothed version will cover them).
        uncovered_positions -= set(range(ref_start, ref_end + 1))
        
        # Smoothed sequence; we'll edit this so that if this read has (mis)matches to any called mutated
        # positions, these positions are updated with the read's aligned nucleotides at these positions.
        smoothed_aln_seq = fasta[ref_start: ref_end + 1]
        
        if actually_include_mutations_in_the_smoothed_reads:
            # just for debugging: track the exact edits made to smoothed_aln_seq
            replacements_made = {}

            ap = aln.get_aligned_pairs(matches_only=True)

            # Iterating through the aligned pairs is expensive. Since read lengths are generally in the thousands
            # to tens of thousands of bp (which is much less than the > 1 million bp length of any bacterial genome),
            # we set things up so that we only iterate through the aligned pairs once. We maintain an integer, mpi,
            # that is a poor man's "pointer" to an index in mutated_positions.

            mpi = 0

            # Go through this aln's aligned pairs. As we see each pair, compare the pair's reference position
            # (refpos) to the mpi-th mutated position (herein referred to as "mutpos").
            #
            # If refpos >  mutpos, increment mpi until refpos <= mutpos (stopping as early as possible).
            # If refpos == mutpos, we have a match! Update readname2mutpos2ismutated[mutpos] based on
            #                      comparing the read to the reference at the aligned positions.
            # If refpos <  mutpos, continue to the next pair.

            for pair in ap:

                refpos = pair[1]
                mutpos = mutated_positions[mpi]

                no_mutations_to_right_of_here = False

                # Increment mpi until we get to the next mutated position at or after the reference pos for this
                # aligned pair (or until we run out of mutated positions).
                while refpos > mutpos:
                    mpi += 1
                    if mpi < len(mutated_positions):
                        mutpos = mutated_positions[mpi]
                    else:
                        no_mutations_to_right_of_here = True
                        break

                # I expect this should happen only for reads aligned near the right end of the genome.
                if no_mutations_to_right_of_here:
                    break

                # If the next mutation occurs after this aligned pair, continue on to a later pair.
                if refpos < mutpos:
                    continue

                # If we've made it here, refpos == mutpos!
                # (...unless I messed something up in how I designed this code.)
                if refpos != mutpos:
                    raise ValueError("This should never happen!")

                # Finally, get the nucleotide aligned to this mutated position from this read.
                readpos = pair[0]
                read_nt = aln.query_sequence[readpos]

                # If this read doesn't have match the first or second most common nucleotide at this position,
                # "smooth" it to just have the most common nucleotide. In the future, when we perform read
                # smoothing based on an arbitrary set of SNV calls, we can be more careful about this; but for now
                # we make the simplifiying assumption that a mutation likely only has one alternate nucleotide,
                # and that the 3rd and 4th most common nucleotides indicate errors.
                # (Also, note that we break ties here arbitrarily.)
                nt2ct = dict(zip("ACGT", mutpos2pileup[mutpos][0]))
                nt1 = max(nt2ct, key=nt2ct.get)
                del nt2ct[nt1]
                nt2 = max(nt2ct, key=nt2ct.get)
                
                if read_nt != nt1 and read_nt != nt2:
                    if no_indoor_voice:
                        print(
                            f"Read {new_readname} has 3rd or 4th most common nt at mutpos {mutpos + 1:,}: "
                            f"pileup = {mutpos2pileup[mutpos]}, read = {read_nt}"
                        )
                    read_nt = nt1
                    
                # Notably, the nucleotide at a mutated position in a smoothed read will always be the first
                # or second most common nucleotide at this position. So "unreasonable" positions, in which
                # the ref nt != the consensus nt, will not be treated as you might expect -- we ignore the
                # reference in this particular case. Shouldn't make a big difference, since in most cases
                # the ref and consensus nt agree.
                relative_pos_on_aln = mutpos - ref_start
                smoothed_aln_seq = smoothed_aln_seq.replace([relative_pos_on_aln], read_nt)
                replacements_made[relative_pos_on_aln] = read_nt
                if no_indoor_voice:
                    print(
                        f"Read {new_readname} mismatches ref at mutpos {mutpos + 1:,}: "
                        f"ref = {ref_nt}, read = {read_nt}"
                    )

            if no_indoor_voice:
                print(f"Read {new_readname} required {len(replacements_made):,} replacements!")
        
        # Now that we've finished processing all called mutations that this alignment spans, prepare it
        # to be written out to a FASTA file. See comments above on smoothed_aln_buffer, and why we don't
        # just write everything out as soon as it's ready.
        #
        # (Also, we've already guaranteed readname isn't already in smoothed_aln_buffer, so no need to worry
        # about accidentally overwriting something from earlier.)
        smoothed_aln_buffer[new_readname] = smoothed_aln_seq
        
        if ai % ALN_BUFFER_FREQ == 0:
            write_out_reads(output_smoothed_reads_file, smoothed_aln_buffer)
            # Clear the buffer
            smoothed_aln_buffer = {}
        
    # We're probably going to have left over smoothed reads that we still haven't written out, unless things
    # worked out so that on the final alignment we saw ai was exactly divisible by ALN_BUFFER_FREQ (and that's
    # pretty unlikely unless you set the buffer freq to a low number). So make one last dump of the buffer.
    if len(smoothed_aln_buffer) > 0:
        write_out_reads(output_smoothed_reads_file, smoothed_aln_buffer)
        
    if len(uncovered_positions) > 0:
        print(f"For reference, there are {len(uncovered_positions):,} uncovered positions in {seq2name[seq]}.")
        
        sup = sorted(uncovered_positions)
        uc_runs = convert_to_runs(sup)
        print(f'And there are {len(uc_runs)} "runs" of uncovered positions.')
        
        rounded_meancov = round(seq2meancov[seq])
        print(
            f'Adding "virtual reads" spanning each of these runs, at rounded mean coverage '
            f'of {rounded_meancov:,}x, to account for this...'
        )
        
        num_vr = 0
        vr_buffer = {}
        for run in uc_runs:
            # Construct a virtual read that includes this entire run of uncovered positions as well
            # as VR_EXTRA_SPAN positions before and after (clamping to the start/end of the seq if needed).
            #
            # Notably, we could try to make this loop around from end -> start if this is a cyclic MAG, but
            # to remain consistent with how we handle supplementary alignments above -- and because implementing
            # the loop around would be a lot of work and it's like 3am -- we ignore this for now.
            #
            # Also, note that run_start can equal run_end, if only a single isolated position is uncovered.
            # This is fine -- the code handles this case automatically. (I guess the only potential problem is
            # if the length of the MAG is less than VR_EXTRA_SPAN, but... that should never happen. If you have
            # like 100bp-long MAGs that's a problem! I guess, TODO, make note of this when generalizing this
            # code.)
            
            run_start = max(run[0] - VR_EXTRA_SPAN, 0)
            run_end = min(run[1] + VR_EXTRA_SPAN, seq2len[seq] - 1)
            
            # Generate a sequence matching the "reference" MAG at these positions. We of course don't have
            # any info about mutations here, because these positions are uncovered by the real reads!
            vr_seq = fasta[run_start: run_end + 1]
            
            # We need to assign reads unique names, and including the run coordinates here is a nice way
            # to preserve uniqueness across runs and also make our smoothed reads files easier to interpret
            vr_name_prefix = f"vr_{run[0]}_{run[1]}"
            
            # Add M copies of this virtual read, where M = (rounded mean coverage of this MAG)
            for vr_num in range(1, rounded_meancov + 1):
                vr_name = f"{vr_name_prefix}_{vr_num}"
                vr_buffer[vr_name] = vr_seq
                num_vr += 1
                
        write_out_reads(output_smoothed_reads_file, vr_buffer)
        print(f"Wrote out {num_vr:,} virtual reads.")
    
    print(f"Done with {seq}! Took {time.time() - ts1:,.2f} sec.")
        
print(f"Time taken: {time.time() - t1:,} sec.")

Identifying mutated positions (p = 1%) in genome CAMP...
Found 83 mutated positions (p = 1%) in CAMP.
Note that this tally is higher than you'd see in e.g. the CP1/2/3 plots, because now we're including both 'rare' and non-rare mutations. Just so you don't waste five minutes sanity-checking this like I did.
Going through these positions...
	On aln 5,000 in seq CAMP. Time spent on CAMP so far: 13.85 sec.
	On aln 10,000 in seq CAMP. Time spent on CAMP so far: 36.32 sec.
	On aln 15,000 in seq CAMP. Time spent on CAMP so far: 57.60 sec.
	On aln 20,000 in seq CAMP. Time spent on CAMP so far: 80.53 sec.
	On aln 25,000 in seq CAMP. Time spent on CAMP so far: 102.50 sec.
	On aln 30,000 in seq CAMP. Time spent on CAMP so far: 123.05 sec.
	On aln 35,000 in seq CAMP. Time spent on CAMP so far: 143.75 sec.
	On aln 40,000 in seq CAMP. Time spent on CAMP so far: 165.21 sec.
	On aln 45,000 in seq CAMP. Time spent on CAMP so far: 186.74 sec.
	On aln 50,000 in seq CAMP. Time spent on CAMP so far: 206.1

	On aln 65,000 in seq BACT1. Time spent on BACT1 so far: 1,208.12 sec.
	On aln 70,000 in seq BACT1. Time spent on BACT1 so far: 1,279.04 sec.
	On aln 75,000 in seq BACT1. Time spent on BACT1 so far: 1,389.03 sec.
	On aln 80,000 in seq BACT1. Time spent on BACT1 so far: 1,465.24 sec.
	On aln 85,000 in seq BACT1. Time spent on BACT1 so far: 1,538.79 sec.
	On aln 90,000 in seq BACT1. Time spent on BACT1 so far: 1,619.42 sec.
	On aln 95,000 in seq BACT1. Time spent on BACT1 so far: 1,702.07 sec.
	On aln 100,000 in seq BACT1. Time spent on BACT1 so far: 1,804.05 sec.
	On aln 105,000 in seq BACT1. Time spent on BACT1 so far: 1,888.61 sec.
	On aln 110,000 in seq BACT1. Time spent on BACT1 so far: 2,009.40 sec.
	On aln 115,000 in seq BACT1. Time spent on BACT1 so far: 2,093.49 sec.
	On aln 120,000 in seq BACT1. Time spent on BACT1 so far: 2,182.65 sec.
	On aln 125,000 in seq BACT1. Time spent on BACT1 so far: 2,303.11 sec.
	On aln 130,000 in seq BACT1. Time spent on BACT1 so far: 2,440.16 sec.

	On aln 335,000 in seq BACT2. Time spent on BACT2 so far: 1,595.55 sec.
	On aln 340,000 in seq BACT2. Time spent on BACT2 so far: 1,616.13 sec.
	On aln 345,000 in seq BACT2. Time spent on BACT2 so far: 1,636.92 sec.
	On aln 350,000 in seq BACT2. Time spent on BACT2 so far: 1,656.19 sec.
	On aln 355,000 in seq BACT2. Time spent on BACT2 so far: 1,676.75 sec.
	On aln 360,000 in seq BACT2. Time spent on BACT2 so far: 1,696.99 sec.
	On aln 365,000 in seq BACT2. Time spent on BACT2 so far: 1,717.63 sec.
	On aln 370,000 in seq BACT2. Time spent on BACT2 so far: 1,737.78 sec.
	On aln 375,000 in seq BACT2. Time spent on BACT2 so far: 1,758.50 sec.
	On aln 380,000 in seq BACT2. Time spent on BACT2 so far: 1,779.06 sec.
	On aln 385,000 in seq BACT2. Time spent on BACT2 so far: 1,799.42 sec.
	On aln 390,000 in seq BACT2. Time spent on BACT2 so far: 1,819.94 sec.
	On aln 395,000 in seq BACT2. Time spent on BACT2 so far: 1,840.12 sec.
	On aln 400,000 in seq BACT2. Time spent on BACT2 so far: 1,860.

## 1.5. Stats about smoothed read lengths

We could have just figured this out while computing the stuff above, but ... I didn't have the foresight to think of this earlier, and I don't want to rerun that stuff for another >1 hour, so we just loop through the FASTA files we just generated quickly

In [9]:
for seq in SEQS:
    read_lengths = []
    
    # Parse a FASTA file -- I stole this code from myself in the Diversity Indices notebook
    with open(f"phasing-data/smoothed-reads/{seq}_smoothed_reads.fasta", "r") as fastafile:

        # Assumes that sequences are not split up over multiple lines (so a FASTA file with N sequences
        # should have only 2N lines, maybe 2N + 1 if there's an extra empty newline at the bottom of the file)
        for linenum, line in enumerate(fastafile):

            if line.startswith(">"):
                if linenum % 2 != 0:
                    raise ValueError("something weird with > location in all_edges.fasta. Go yell at Marcus.")
            else:
                if linenum % 2 != 1:
                    raise ValueError("something weird with non > location in all_edges.fasta. Go yell at Marcus.")

                read_lengths.append(len(line.strip()))

    num_reads = len(read_lengths)
    minlen = min(read_lengths)
    maxlen = max(read_lengths)
    avglen = mean(read_lengths)
    medlen = median(read_lengths)
    
    # Reads with length less than w + k = (threshold) will be ignored by jumboDB when constructing
    # the graph, so we output stats about this to verify that we're not dropping a TON of reads
    # (ofc ideally we wouldn't drop any tho...)
    threshold = 7001
    geq_threshold = len([rl for rl in read_lengths if rl >= threshold])
    pct = 100 * (geq_threshold / num_reads)
    
    print(f"{seq2name[seq]}: {num_reads:,} smoothed reads")
    print(f"\tmin / mean / median / max length = {minlen:,} / {avglen:,.1f} / {medlen:,} / {maxlen:,}")
    print(f"\tNum of reads with length \u2265 {threshold:,}: {geq_threshold:,} / {num_reads:,} ({pct:.1f}%)")

CAMP: 476,356 smoothed reads
	min / mean / median / max length = 200 / 11,264.7 / 11,154.0 / 35,099
	Num of reads with length ≥ 7,001: 438,334 / 476,356 (92.0%)
BACT1: 265,975 smoothed reads
	min / mean / median / max length = 202 / 11,487.2 / 11,473 / 39,723
	Num of reads with length ≥ 7,001: 243,480 / 265,975 (91.5%)
BACT2: 742,926 smoothed reads
	min / mean / median / max length = 200 / 11,318.0 / 11,259.0 / 40,346
	Num of reads with length ≥ 7,001: 673,220 / 742,926 (90.6%)


## 2. Construct de Bruijn graph(s) from these smoothed reads

Possible TODO: adjust this to detect the options above (which mutations are we using? did we actually include mutations in the smoothed reads?) and adjust where files get spit out to accordingly

### 2.1. jumboDBG

#### 2.1.1. Run jumboDBG

In [10]:
!/home/mfedarko/software/LJA/bin/jumboDBG \
    --reads phasing-data/smoothed-reads/edge_6104_smoothed_reads.fasta \
    -k 5001 \
    --coverage \
    --output-dir phasing-data/smoothed-reads/edge_6104_jumbo_20220221

!/home/mfedarko/software/LJA/bin/jumboDBG \
    --reads phasing-data/smoothed-reads/edge_1671_smoothed_reads.fasta \
    -k 5001 \
    --coverage \
    --output-dir phasing-data/smoothed-reads/edge_1671_jumbo_20220221

!/home/mfedarko/software/LJA/bin/jumboDBG \
    --reads phasing-data/smoothed-reads/edge_2358_smoothed_reads.fasta \
    -k 5001 \
    --coverage \
    --output-dir phasing-data/smoothed-reads/edge_2358_jumbo_20220221

00:00:00 14.5Gb  INFO: Hello! You are running jumboDBG, a tool for construction of de Bruijn graphs for arbitrarily large values of k
00:00:00 14.5Gb  INFO: Note that jumboDBG does not perform any error correction and ignores all reads shorter than k + w = 7001
00:00:00 0Mb  INFO: Reading reads
00:00:00 0Mb  INFO: Extracting minimizers
00:00:28 3.3Gb  INFO: Finished read processing
00:00:28 3.3Gb  INFO: 2541151 hashs collected. Starting sorting.
00:00:28 3.4Gb  INFO: Finished sorting. Total distinct minimizers: 1695
00:00:28 3.4Gb  INFO: Starting construction of sparse de Bruijn graph
00:00:28 3.4Gb  INFO: Vertex map constructed.
00:00:28 3.4Gb  INFO: Filling edge sequences.
00:00:55 4.5Gb  INFO: Finished sparse de Bruijn graph construction.
00:00:55 4.5Gb  INFO:  Collecting tips 
00:00:55 4.5Gb  INFO: Added 58 artificial minimizers from tips.
00:00:55 4.5Gb  INFO: Collected 3498 old edges.
00:00:55 4.5Gb  INFO: New minimizers added to sparse graph.
00:00:55 4.5Gb  INFO: Refilling grap

#### 2.1.2. Filter jumboDBG graphs to remove low-coverage edges

In [27]:
# yanked and slightly modified from main-workflow/utils.py
import networkx as nx

def load_gfa(filepath):
    
    # We ignore directionality for right now.
    graph = nx.Graph()

    with open(filepath, "r") as gfafile:
        for line in gfafile:
            if line[0] == "S":
                parts = line.strip().split("\t")
                node_name = parts[1]
                node_len = len(parts[2])
                node_cov = None

                # Parse GFA tags
                extra_data = parts[3:]
                for tag in extra_data:
                    # This is overcautious, probably, but whatevs
                    if tag.startswith("LN:i:"):
                        raise ValueError("Duplicate length for node {}".format(node_name))
                    elif tag.startswith("dp:i:") or tag.startswith("KC:i:"):
                        if node_cov is None:
                            node_cov = int(tag[5:])
                        else:
                            raise ValueError("Duplicate coverage for node {}".format(node_name))

                if node_cov is None:
                    raise ValueError("No coverage tag given for node {}".format(node_name))

                graph.add_node(node_name, length=node_len, cov=node_cov)

            elif line[0] == "L":
                parts = line.strip().split("\t")
                src = parts[1]
                snk = parts[3]
                src_orient = parts[2]
                snk_orient = parts[4]
                graph.add_edge(src, snk, src_orient=src_orient, snk_orient=snk_orient)
    return graph

In [37]:
MIN_EDGE_COV = 5
for seq in SEQS:
    phdir = f"phasing-data/smoothed-reads/{seq}_jumbo_20220221"
    g = load_gfa(os.path.join(phdir, "graph.gfa"))
    removed_nodes = []
    for n in list(g.nodes):
        if g.nodes[n]["cov"] < MIN_EDGE_COV:
            g.remove_node(n)
            removed_nodes.append(n)
    print(f"{seq2name[seq]}: Removed {len(removed_nodes):,} node(s) with coverage < {MIN_EDGE_COV}x.")

    # Probs possible to merge this together with the filtering step above so we only loop over the nodes once
    with open(os.path.join(phdir, "graph-covfilt.gfa"), "w") as f:
        f.write("H\tVN:Z:1.0\n")
        for n in g.nodes:
            f.write(f"S\t{n}\t*\tKC:i:{g.nodes[n]['cov']}\tLN:i:{g.nodes[n]['length']}\n")
        for e in g.edges:
            src_or = g.edges[e]["src_orient"]
            snk_or = g.edges[e]["snk_orient"]
            f.write(f"L\t{e[0]}\t{src_or}\t{e[1]}\t{snk_or}\t*\n")

CAMP: Removed 1 node(s) with coverage < 5x.
BACT1: Removed 14 node(s) with coverage < 5x.
BACT2: Removed 4 node(s) with coverage < 5x.


### 2.2. LJA (including jumboDBG and multiplexDBG)

`--noec` is an undocumented flag that avoids the error correction step in LJA, which is what we want.

In [8]:
!/home/mfedarko/software/LJA/bin/lja \
    --reads phasing-data/smoothed-reads/edge_6104_smoothed_reads.fasta \
    --noec \
    --output-dir phasing-data/smoothed-reads/edge_6104_lja_noec

!/home/mfedarko/software/LJA/bin/lja \
    --reads phasing-data/smoothed-reads/edge_1671_smoothed_reads.fasta \
    --noec \
    --output-dir phasing-data/smoothed-reads/edge_1671_lja_noec

!/home/mfedarko/software/LJA/bin/lja \
    --reads phasing-data/smoothed-reads/edge_2358_smoothed_reads.fasta \
    --noec \
    --output-dir phasing-data/smoothed-reads/edge_2358_lja_noec

00:00:00 14.5Gb  INFO: Hello! You are running La Jolla Assembler (LJA), a tool for genome assembly from PacBio HiFi reads
00:00:00 14.5Gb  INFO: 02ec23dd81b81ce8718a70fb64ea6fab0d88c5ca
00:00:00 14.5Gb  INFO: LJA pipeline started
00:00:00 14.5Gb  INFO: Performing initial correction with k = 5001
00:00:00 0Mb  INFO: Reading reads
00:00:00 0Mb  INFO: Extracting minimizers
00:00:17 4.3Gb  INFO: Finished read processing
00:00:17 4.3Gb  INFO: 5257129 hashs collected. Starting sorting.
00:00:17 4.3Gb  INFO: Finished sorting. Total distinct minimizers: 4475
00:00:17 4.3Gb  INFO: Starting construction of sparse de Bruijn graph
00:00:17 4.3Gb  INFO: Vertex map constructed.
00:00:17 4.3Gb  INFO: Filling edge sequences.
00:00:34 4.3Gb  INFO: Finished sparse de Bruijn graph construction.
00:00:34 4.3Gb  INFO:  Collecting tips 
00:00:34 4.3Gb  INFO: Added 60 artificial minimizers from tips.
00:00:34 4.3Gb  INFO: Collected 8998 old edges.
00:00:34 4.3Gb  INFO: New minimizers added to sparse graph.
0

00:04:23 0.5Gb  INFO: Export to Dot
00:04:23 0.5Gb  INFO: Export to GFA and compressed contigs
00:04:28 0.5Gb  INFO: Finished repeat resolution
00:04:29 14.5Gb  INFO: Performing polishing and homopolymer uncompression
00:04:31 51Mb  INFO: Aligning reads back to assembly
00:04:52 3.4Gb  INFO: Finished alignment.
00:04:52 3.4Gb  INFO: Printing alignments to "phasing-data/smoothed-reads/edge_1671_lja_noec/uncompressing/alignments.txt"
00:04:56 4.6Gb  INFO: Reading and processing initial reads from ["phasing-data/smoothed-reads/edge_1671_smoothed_reads.fasta"]
00:09:07 5.8Gb  INFO: Uncompressing homopolymers in contigs
00:09:13 5.8Gb  INFO: Total zero covered nucleotides 62090
00:09:13 5.8Gb  INFO: Calculating overlaps between adjacent uncompressed edges
00:09:16 5.8Gb  INFO: Printing final gfa file to "phasing-data/smoothed-reads/edge_1671_lja_noec/mdbg.gfa"
00:09:17 5.8Gb  INFO: Printing final assembly to "phasing-data/smoothed-reads/edge_1671_lja_noec/assembly.fasta"
00:09:17 14.5Gb  IN